<a href="https://colab.research.google.com/github/sclatch1/recommenderSystems/blob/main/dataexploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# importing libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.sparse import csr_matrix
import scipy as sp

from recpack.matrix import InteractionMatrix
from recpack.scenarios import WeakGeneralization
from recpack.util import get_top_K_ranks
from recpack.preprocessing.filters import MinItemsPerUser, MinUsersPerItem
from ydata_profiling import ProfileReport

# from src.metrics import calculate_ndcg, calculate_calibrated_recall



In [3]:
ROOT_DIR = Path.cwd().parent

data_dir_path = ROOT_DIR / "data"

## Datastructure
    - type of data: catergorical, numerical, text, dates
    - remove duplicates and irrelevan features
    - datatypes align with the content ?

In [4]:
games = pd.read_csv(f"{data_dir_path}/games.csv")
user_reviews = pd.read_csv(f"{data_dir_path}/user_reviews.csv")
train_interaction = pd.read_csv(f"{data_dir_path}/train_interactions.csv")
extended_games = pd.read_csv(f"{data_dir_path}/extended_games.csv")
test_interaction = pd.read_csv(f"{data_dir_path}/test_interactions_in.csv")


In [2]:
# Looking for relevant columns

print("games columns")
print(games.columns)
print("user_reviews columns")
print(user_reviews.columns)
print("train interactions columns")
print(train_interaction.columns)
print("extended games columns")
print(extended_games.columns)

games columns


NameError: name 'games' is not defined

In [5]:
s1 = pd.merge(test_interaction, train_interaction, how='inner', on=['user_id'])
print(f"Number of overlapping users in train and test interactions: {s1['user_id'].nunique()}")


Number of overlapping users in train and test interactions: 0


In [5]:
print(test_interaction.head())

   user_id  item_id                     item_name  playtime
0        4      760                 Ace of Spades        83
1        4     8417  Geometry Wars: Retro Evolved        18
2        4     5238      Half-Life 2: Episode One       491
3        4     8250                         Shank        13
4        4     8278               BIT.TRIP RUNNER        74


We want to evaluate ItemKNN, which is based on item-user interaction. So we will not use the following datasets: games, extended games.

In [6]:
# remove the duplicates record
train_interaction.drop_duplicates(inplace=True)

In [7]:
print(train_interaction.describe().T)

              count          mean           std  min      25%      50%  \
user_id   2267397.0  25482.816028  17377.091397  0.0  10886.0  23223.0   
item_id   2267397.0   2994.252530   2873.162816  0.0    678.0   1653.0   
playtime  2267397.0   1577.700248   6890.524187  1.0     46.0    210.0   

              75%       max  
user_id   37033.0   68402.0  
item_id    5240.0    8522.0  
playtime    812.0  635295.0  


In [9]:
train_interaction['data'].boxplot(meanline=True, showmeans=True)

KeyError: 'data'

In [ ]:

profile = ProfileReport(train_interaction, title="Steam Dataset Report", explorative=True)
profile.to_file(f"{ROOT_DIR}/reports/train_interaction_report")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:04<00:00,  1.17s/it]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

/home/david/recommenderSystems/.venv/lib/python3.11/site-packages/ydata_profiling/profile_report.py:386: UserWarning: Extension  not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  warnings.warn(


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
user_reviews.drop_duplicates(inplace=True)
user_reviews.head()

,user_id,item_id,funny,posted,last_edited,helpful,recommend,review
0,0,140,NaN,"Posted November 5, 2011.",NaN,No ratings yet,True,Simple yet with great replayability. In my opi...
1,0,711,NaN,"Posted July 15, 2011.",NaN,No ratings yet,True,It's unique and worth a playthrough.
2,1,815,NaN,"Posted September 8, 2013.",NaN,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
3,1,1027,NaN,"Posted November 29, 2013.",NaN,1 of 4 people (25%) found this review helpful,True,Very fun little game to play when your bored o...
4,2,1224,NaN,Posted February 3.,NaN,No ratings yet,True,A suitably punishing roguelike platformer. Wi...


We can remove the following collumns review, helpful, funny and last edited

In [11]:
user_reviews.drop(columns=['posted', 'helpful', 'funny', 'review', 'last_edited'], inplace=True)

In [ ]:
profile = ProfileReport(user_reviews, title="Steam Dataset Report", explorative=True)
profile.to_file(f"{ROOT_DIR}/reports/user_reviews_report")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 72.68it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

/home/david/recommenderSystems/.venv/lib/python3.11/site-packages/ydata_profiling/profile_report.py:386: UserWarning: Extension  not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  warnings.warn(


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
print(user_reviews.head())

   user_id  item_id  recommend
0        0      140       True
1        0      711       True
2        1      815       True
3        1     1027       True
4        2     1224       True


In [10]:
print(train_interaction.head())
train_interaction = train_interaction.drop(columns=["item_name"])
test_interaction = test_interaction.drop(columns=["item_name"])

   user_id  item_id               item_name  playtime
0        0        0          Counter-Strike         6
1        0     2555           Day of Defeat         7
2        0     2556   Day of Defeat: Source      4733
3        0     1043  Counter-Strike: Source      1853
4        0     5335             Psychonauts       333


In [11]:
train_interaction = pd.concat([train_interaction, test_interaction], ignore_index=True)
train_interaction = train_interaction.drop_duplicates(subset=['user_id', 'item_id'])

print(train_interaction.head())

   user_id  item_id  playtime
0        0        0         6
1        0     2555         7
2        0     2556      4733
3        0     1043      1853
4        0     5335       333


In [1]:
print(train_interaction.describe().T)

NameError: name 'train_interaction' is not defined

In [7]:
print(train_interaction.shape)
print(test_interaction.shape)

(2293985, 4)
(448211, 4)


In [26]:
from ydata_profiling import ProfileReport
profile = ProfileReport(train_interaction, title="Steam Dataset Report", explorative=True)
profile.to_file(f"{ROOT_DIR}/reports/train_interaction_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:01<00:00,  2.06it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [37]:
train_interaction = train_interaction[train_interaction['playtime'] >= 300]
train_interaction['implicit'] = 1
train_interaction = train_interaction[['user_id', 'item_id', 'implicit']]

In [38]:
train_interaction.head()
train_interaction.to_csv(f"{data_dir_path}/train_interactions_cleaned.csv", index=False)

In [32]:


user_reviews['recommend'] = user_reviews['recommend'].map({True: 1, False: -1})

print(user_reviews['recommend'].value_counts())


df_merged = pd.merge(train_interaction, user_reviews, on=['user_id', 'item_id'], how='outer')
df_merged['playtime'] = df_merged['playtime'].fillna(df_merged['recommend'])
df_merged = df_merged.drop(columns=['recommend'])
print(df_merged.head(50))


Series([], Name: count, dtype: int64)
    user_id  item_id  playtime
0         0        0       6.0
1         0        9      25.0
2         0       11    1886.0
3         0       31      44.0
4         0       33     186.0
5         0       39     772.0
6         0       43     640.0
7         0       69    1204.0
8         0       90    1494.0
9         0      140   10006.0
10        0      162     115.0
11        0      205     110.0
12        0      206    2085.0
13        0      207      28.0
14        0      250      31.0
15        0      255    4431.0
16        0      256    1613.0
17        0      264     594.0
18        0      273     126.0
19        0      302     100.0
20        0      307    3478.0
21        0      310     459.0
22        0      329     599.0
23        0      355      18.0
24        0      371     241.0
25        0      389    1429.0
26        0      401      69.0
27        0      450     798.0
28        0      462      46.0
29        0      479     139.0
3

In [11]:
df_merged.fillna({'recommend': 0}, inplace=True)

print(df_merged.value_counts())

print((df_merged['recommend'] == 1).sum())

percentage_recommended = ((df_merged['recommend'] == 1) | (df_merged['recommend'] == -1)).mean() * 100
print(f"Percentage of recommended games in training interactions: {percentage_recommended:.2f}%")
print(df_merged.head())


user_id  item_id  item_name                         playtime  recommend
1907     650      PAYDAY 2                          9769      1.0          3
1172     8363     Counter-Strike: Global Offensive  71558     1.0          3
1907     1538     The Walking Dead: Season Two      1491      1.0          3
12828    39       Terraria                          24297     1.0          3
         1653     Subnautica                        2876      1.0          3
                                                                          ..
14470    5165     Fallout 4                         3544      0.0          1
         5238     Half-Life 2: Episode One          1         0.0          1
         5635     WARMODE                           695       0.0          1
         5888     Portal                            240       0.0          1
         3935     Destination Sol                   54        0.0          1
Name: count, Length: 2267397, dtype: int64
29190
Percentage of recommended games 

In [33]:

print(f"train interaction shape: {train_interaction.shape}")
print(f"df_merged shape: {df_merged.shape}")

train interaction shape: (2267397, 3)
df_merged shape: (2277264, 3)


In [37]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df_merged, title="Steam Dataset Report", explorative=True)
profile.to_file(f"{ROOT_DIR}/reports/final_df_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:01<00:00,  1.69it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
print(df_merged.head())
df_merged.drop(columns=["item_name"])

   user_id  item_id               item_name  playtime  recommend
0        0        0          Counter-Strike         6        0.0
1        0     2555           Day of Defeat         7        0.0
2        0     2556   Day of Defeat: Source      4733        0.0
3        0     1043  Counter-Strike: Source      1853        0.0
4        0     5335             Psychonauts       333        0.0


,user_id,item_id,playtime,recommend
0,0,0,6,0.0
1,0,2555,7,0.0
2,0,2556,4733,0.0
3,0,1043,1853,0.0
4,0,5335,333,0.0
...,...,...,...,...
2267890,68401,6378,8,0.0
2267891,68402,2707,677,0.0
2267892,68402,827,43,0.0
2267893,68402,5544,3,0.0


In [38]:
df_merged.to_csv("merged.csv")

In [47]:
# new pipeline baseline

import torch
from hyperopt import hp
import numpy as np

# from recpack.algorithms import ItemKNN, BPRMF
from recpack.pipelines import PipelineBuilder, HyperoptInfo
from recpack.scenarios import WeakGeneralization
from recpack.datasets import DummyDataset

from os import chdir, getcwd


# print(getcwd())

# chdir("recommenderSystems")

# print(getcwd())

import src.mfbpr as m

import importlib
importlib.reload(m)
torch.serialization.add_safe_globals([m])




optimisation_info = HyperoptInfo(
    hp.choice(
        'bprmf', [
            {
                'num_components': 100,
                "learning_rate" : 0.01,
                "lambda_h": 0.001,
                "lambda_w": 0.001,
                "validation_sample_size": 100,
                "predict_topK": 100,
                "save_best_to_file": True,
                "keep_last": False,
            }
        ]
    ),
    max_evals=50,
)


im = DummyDataset().load()

X = InteractionMatrix(df_merged, item_ix="item_id", user_ix="user_id")

scenario = WeakGeneralization(validation=True)
scenario.split(im)

pb = PipelineBuilder()
# Use the configured search space when optimising the ItemKNN parameters
pb.add_algorithm(m.BPRMF, optimisation_info=optimisation_info)
pb.set_data_from_scenario(scenario)
# Set NDCG@10 as the optimisation metric.
# Since it is a metric to be maximized, the loss will be the negative of the NDCG, such that hyperopt can minimize it.
pb.set_optimisation_metric('NDCGK', 10)
pb.add_metric('NDCGK', 10)

pipe = pb.build()
pipe.run()

df =pd.DataFrame.from_dict(pipe.get_metrics())
df.head()

  0%|          | 0/496 [00:00<?, ?it/s]

  0%|          | 0/496 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

train_epoch BPRMF: 0it [00:00, ?it/s]

2025-12-07 18:07:01,963 - base - recpack - INFO - Processed epoch 19 in 0.02 s.Batch Training Loss = 0.6244
2025-12-07 18:07:01,974 - stopping_criterion - recpack - INFO - StoppingCriterion has value 0.7045783675276917, which is worse than previous iterations.
2025-12-07 18:07:01,974 - base - recpack - INFO - Evaluation at end of 19 took 0.01 s.


job exception: module 'src.mfbpr' has no attribute '__module__'



  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]


AttributeError: module 'src.mfbpr' has no attribute '__module__'

In [50]:
df =pd.DataFrame.from_dict(pipe.get_metrics())
print(df.head())

Empty DataFrame
Columns: []
Index: []


Let's first build an interactionMatrix with train_interaction dataset

Weak generalization is more practical. For some algorithms (e.g. matrix
factorization), a strong generalization data split is infeasible as the
algorithm relies on learned user representations.

In [ ]:
# fix seed for reproducibility
scenario = WeakGeneralization(frac_data_in=0.8, validation=True, seed=42)
scenario.split(X)


X_train = scenario.full_training_data.values
X_test_in = scenario.test_data_in.values
X_test_out = scenario.test_data_out.values

X_val_train, X_val_target = scenario.validation_data
val_train = X_val_train.values
val_target = X_val_target.values

df_test_out = matrix2df(X_test_out)

NameError: name 'WeakGeneralization' is not defined

In [ ]:
def filter_sparse_rows_min_interactions(X, min_interactions=5):
    """Keep only rows with at least min_interactions non-zero entries."""
    counts = np.diff(X.indptr)
    valid_rows = np.where(counts >= min_interactions)[0]
    return X[valid_rows]
X_test_in = filter_sparse_rows_min_interactions(X_test_in, min_interactions=5)
X_test_out = filter_sparse_rows_min_interactions(X_test_out, min_interactions=5)
val_train = filter_sparse_rows_min_interactions(val_train, min_interactions=5)
val_target = filter_sparse_rows_min_interactions(val_target, min_interactions=5)  

NameError: name 'X_test_in' is not defined

In [ ]:
# Interactions per user in test set
test_counts = X_test_out.sum(axis=1)
test_counts = np.diff(X_test_out.indptr)
print("Median test interactions per user:", np.median(test_counts))
print("Min test interactions per user:", test_counts.min())
print("Max test interactions per user:", test_counts.max())
print("Mean test interactions per user:", test_counts.mean())

cold_1 = np.sum(test_counts < 1)
cold_5 = np.sum(test_counts < 5)
cold_10 = np.sum(test_counts < 10)

print("===== Cold-User Counts =====")
print(f"Users with < 1 interaction:      {cold_1}")
print(f"Users with < 5 interactions:     {cold_5}")
print(f"Users with < 10 interactions:    {cold_10}")


Median test interactions per user: 3.0
Min test interactions per user: 0
Max test interactions per user: 579
Mean test interactions per user: 6.3142552227241495
===== Cold-User Counts =====
Users with < 1 interaction:      21690
Users with < 5 interactions:     40441
Users with < 10 interactions:    53178


In [ ]:
import os
os.chdir("/home/david/recommenderSystems")
import importlib
from src import mfbpr

importlib.reload(mfbpr)

from src.mfbpr import PPAC_BPRMF

model = PPAC_BPRMF(
    num_components=100,
    learning_rate=0.01,
    lambda_w=0.001,
    lambda_h=0.001,
    validation_sample_size=100,
    predict_topK=100,
    save_best_to_file=f"{ROOT_DIR}/models/best_model.pkl"
)

model._init_model(X_train)



Initializing PPAC_BPRMF
1000


In [ ]:
model.fit(X_train, validation_data=(val_train, val_target))


1000


/home/david/recommenderSystems/src/mfbpr.py:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pos_w = torch.tensor(
/home/david/recommenderSystems/src/mfbpr.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  neg_w = torch.tensor(


2025-12-03 21:37:16,594 - base - recpack - INFO - Processed epoch 0 in 7.88 s.Batch Training Loss = 0.0173


/home/david/recommenderSystems/.venv/lib/python3.11/site-packages/scipy/sparse/_index.py:201: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray_sparse(i, j, x)


2025-12-03 21:37:17,563 - stopping_criterion - recpack - INFO - StoppingCriterion has value 0.6931471816741577, which is better than previous iterations.
2025-12-03 21:37:17,564 - base - recpack - INFO - Model improved. Storing better model.
2025-12-03 21:37:17,595 - base - recpack - INFO - Evaluation at end of 0 took 1.00 s.
2025-12-03 21:37:25,517 - base - recpack - INFO - Processed epoch 1 in 7.92 s.Batch Training Loss = 0.0157
2025-12-03 21:37:26,513 - stopping_criterion - recpack - INFO - StoppingCriterion has value 0.6931471811025262, which is worse than previous iterations.
2025-12-03 21:37:26,514 - base - recpack - INFO - Evaluation at end of 1 took 1.00 s.
2025-12-03 21:37:34,853 - base - recpack - INFO - Processed epoch 2 in 8.34 s.Batch Training Loss = 0.0157
2025-12-03 21:37:36,305 - stopping_criterion - recpack - INFO - StoppingCriterion has value 0.6931471809356334, which is worse than previous iterations.
2025-12-03 21:37:36,306 - base - recpack - INFO - Evaluation at en

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL recpack.algorithms.bprmf.MFModule was not an allowed global by default. Please use `torch.serialization.add_safe_globals([recpack.algorithms.bprmf.MFModule])` or the `torch.serialization.safe_globals([recpack.algorithms.bprmf.MFModule])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.lo

In [ ]:
scores = model.predict(X_test_in)


In [ ]:
df_recos = scores2recommendations(scores, X_test_in, 10)

df_recos.drop(columns=['rank'], inplace=True)
df_recos.to_csv(f"{ROOT_DIR}/reports/df_recos.csv", index=False)


/home/david/recommenderSystems/.venv/lib/python3.11/site-packages/scipy/sparse/_index.py:210: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
from src.metrics import calculate_ndcg, calculate_calibrated_recall
ndcg = calculate_ndcg(df_recos, 10, df_test_out)
recall = calculate_calibrated_recall(df_recos, 10, df_test_out)

print(f"  NDCG@10: {ndcg:.5f}")
print(f"Recall@10: {recall:.5f}")

  NDCG@10: 0.00012
Recall@10: 0.00019


In [ ]:
import numpy as np
user_interactions = np.diff(X_train.tocsr().indptr)
print("Mean interactions per user:", user_interactions.mean())
print("Median:", np.median(user_interactions))
print("Users with < 5 interactions:", (user_interactions < 5).sum())


Mean interactions per user: 26.535655276279044
Median: 13.0
Users with < 5 interactions: 23022


In [ ]:

# ---------------------------
# 1. Compute "item length" baseline (median playtime per item)
# ---------------------------
item_median = train_interaction.groupby("item_id")["playtime"].median()
df = train_interaction.merge(item_median.rename("item_median"), on="item_id", how="left")

# Avoid divide-by-zero
df["item_median"].replace(0, np.nan, inplace=True)
df["item_median"].fillna(df["item_median"].median(), inplace=True)

# ---------------------------
# 2. Normalize playtime by typical item duration
# ---------------------------
df["playtime_norm"] = df["playtime"] / df["item_median"]

# ---------------------------
# 3. Apply log transform
# ---------------------------
df["weight"] = np.log1p(df["playtime_norm"])

/tmp/ipython-input-4134867206.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["item_median"].replace(0, np.nan, inplace=True)
/tmp/ipython-input-4134867206.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 

In [ ]:
# Keep category dtypes
user_cat = df["user_id"].astype("category")
item_cat = df["item_id"].astype("category")

df["user_idx"] = user_cat.cat.codes
df["item_idx"] = item_cat.cat.codes

num_users = len(user_cat.cat.categories)
num_items = len(item_cat.cat.categories)

# Build TRAIN matrix
matrix = csr_matrix(
    (df["weight"], (df["user_idx"], df["item_idx"])),
    shape=(num_users, num_items)
)

In [ ]:
from src.mfbpr import PPAC_BPRMF

model = PPAC_BPRMF(
    num_components=128,
    batch_size=1000,
    max_epochs=10,
    learning_rate=0.001,
    gamma=0.1,  # Weight for local popularity
    beta=0.1,   # Weight for global popularity
    reg_coe=0.001,  # Popularity predictor regularization
    l2_coe=1e-4     # Embedding regularization
)

model.fit(matrix)

2025-11-25 14:28:21,993 - base - recpack - INFO - Fitting ItemKNN complete - Took 6.84s


INFO:recpack:Fitting ItemKNN complete - Took 6.84s


ItemKNN()

In [ ]:
# ---------------------------
# TEST DATA
# ---------------------------
test = test_interaction.copy()

# Merge training medians into test
test = test.merge(item_median.rename("item_median"), on="item_id", how="left")

# For unseen items in test: use global median (keeps transformation stable)
fallback = item_median.median()
test["item_median"] = test["item_median"].fillna(fallback)

# Normalize playtime
test["playtime_norm"] = test["playtime"] / test["item_median"]

# Log transform
test["weight"] = np.log1p(test["playtime_norm"])


In [ ]:
test_to_predict = test_interaction[["user_id", "item_id"]].copy()
test_to_predict.to_csv("test_to_predict.csv", index=False)




In [ ]:
# Encode users and items using training categories
test_to_predict["user_idx"] = (
    test_to_predict["user_id"]
    .astype("category")
    .cat.set_categories(user_cat.cat.categories)
    .cat.codes
)

test_to_predict["item_idx"] = (
    test_to_predict["item_id"]
    .astype("category")
    .cat.set_categories(item_cat.cat.categories)
    .cat.codes
)

# Remove completely unseen users/items
test_to_predict = test_to_predict[
    (test_to_predict["user_idx"] != -1) &
    (test_to_predict["item_idx"] != -1)
]

# Build prediction mask (values = 1)
prediction_matrix = csr_matrix(
    (np.ones(len(test_to_predict)),
     (test_to_predict["user_idx"], test_to_predict["item_idx"])),
    shape=(num_users, num_items)
)



In [ ]:
pred = itemknn.predict(prediction_matrix)
print(pred)


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 0 stored elements and shape (54315, 8368)>


In [ ]:
# 1️⃣ Convert CSR to COO (easy for iteration)
coo = pred.tocoo()

# 2️⃣ Map indices back to original IDs
user_ids = [user_mapping[i] for i in coo.row]
item_ids = [item_mapping[i] for i in coo.col]
values = coo.data

# 3️⃣ Build a DataFrame
df_pred = pd.DataFrame({
    "user_id": user_ids,
    "item_id": item_ids,
    "score": values
})

df_pred.head()


,user_id,item_id,score
